In [70]:
!pip install mp-api --upgrade
import numpy as np
from mp_api.client import MPRester
#from pymatgen.ext.matproj import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure.plotter import BSPlotter
from pymatgen.phonon.plotter import PhononBSPlotter
from jupyter_jsmol.pymatgen import quick_view
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors
from plotly.subplots import make_subplots
import plotly.graph_objects as go

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [71]:
mp_key = "XJhzTsEnBSh1B3Uh2ARw9cKQZuE9Q90J"
mp_id = "mp-569779"

In [72]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
    ph_bs = m.get_phonon_bandstructure_by_material_id(mp_id)
    ph_dos = m.get_phonon_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving PhononBSDOSDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Structure symmetry

In [73]:
i_atom = 7
i_symmop = 2

view = quick_view(prim_struc, "packed", conventional = True)
display(view)
view.script('select antimony; color lightsalmon')
view.script('select sodium; color gold')
view.script('select lithium; color palegreen')
view.script('draw SYMOP ' + str(i_symmop) + ' {atomno = ' + str(i_atom) + '}')

JsmolView(layout=Layout(align_self='stretch', height='400px'))

In [74]:
symmop = symmops[i_symmop - 1]
print(symmop)

Rot:
[[ 0.  1.  0.]
 [-1.  0.  0.]
 [ 0.  0. -1.]]
tau
[0. 0. 0.]


In [75]:
pos_init = conv_struc.sites[i_atom -1].frac_coords
print(pos_init)

[0.5 0.  0.5]


In [76]:
pos_final = symmop.operate(pos_init)
print(pos_final)

[ 0.  -0.5 -0.5]


In [77]:
for k_atom, site in enumerate(conv_struc.sites):
    if np.linalg.norm((site.frac_coords - pos_final)%1) < 1e-6:
        j_atom = k_atom + 1
print(j_atom, conv_struc.sites[j_atom -1].frac_coords)

6 [0.  0.5 0.5]


# Brillouin zone

In [49]:
plot_brillouin_zone(el_bs.structure)

# Electronic bandstructure

In [78]:
fig_el_bs = get_plot_bs(el_bs, plot_range=[-4,7])
fig_el_bs.show()

In [79]:
fig_el_dos = get_plot_dos(el_dos, plot_range=[-4,7])
fig_el_dos.show()

In [80]:
fig_el_bs_and_dos = get_plot_bs_and_dos(el_bs, el_dos, plot_range=[-4,7])
fig_el_bs_and_dos.show()

In [81]:
xvals = fig_el_bs.to_dict()["data"][0]["x"]
yvals_vbm = fig_el_bs.to_dict()["data"][2]["y"]
yvals_cbm = fig_el_bs.to_dict()["data"][3]["y"]

In [82]:
xvals_band_edges = []
yvals_band_edges = []
for i in el_bs.get_vbm()["kpoint_index"]:
    xvals_band_edges.append(xvals[i])
    yvals_band_edges.append(yvals_vbm[i])
for i in el_bs.get_cbm()["kpoint_index"]:
    xvals_band_edges.append(xvals[i])
    yvals_band_edges.append(yvals_cbm[i])

In [83]:
for i in el_bs.get_vbm()["kpoint_index"]:
    scatter = go.Scatter(
        x = xvals_band_edges, y = yvals_band_edges,
        mode = "markers", marker = dict(color="black"),
        showlegend=False)
    fig_el_bs.add_trace(scatter)
fig_el_bs.update_layout(xaxis_range = [xvals[0], xvals[-1]])
fig_el_bs.show()

In [84]:
el_bs.get_vbm()

{'band_index': defaultdict(list, {<Spin.up: 1>: [10, 11, 12]}),
 'kpoint_index': [0, 292, 293],
 'kpoint': <pymatgen.electronic_structure.bandstructure.Kpoint at 0x7f45ec70b610>,
 'energy': 6.5885,
 'projections': {<Spin.up: 1>: array([[0.000e+00, 0.000e+00, 0.000e+00],
         [1.000e-04, 1.680e-02, 2.670e-02],
         [1.000e-04, 1.700e-02, 2.690e-02],
         [1.000e-04, 1.470e-02, 2.330e-02],
         [1.852e-01, 4.900e-03, 2.400e-03],
         [1.603e-01, 4.200e-03, 2.100e-03],
         [0.000e+00, 0.000e+00, 0.000e+00],
         [1.834e-01, 4.800e-03, 2.400e-03],
         [0.000e+00, 0.000e+00, 0.000e+00]])}}

In [85]:
el_bs.get_cbm()

{'band_index': defaultdict(list, {<Spin.up: 1>: [13]}),
 'kpoint_index': [98],
 'kpoint': <pymatgen.electronic_structure.bandstructure.Kpoint at 0x7f45ec9657c0>,
 'energy': 6.604,
 'projections': {<Spin.up: 1>: array([[0.000e+00, 0.000e+00, 3.360e-02],
         [1.600e-03, 1.970e-02, 0.000e+00],
         [0.000e+00, 0.000e+00, 0.000e+00],
         [0.000e+00, 0.000e+00, 0.000e+00],
         [0.000e+00, 0.000e+00, 0.000e+00],
         [0.000e+00, 0.000e+00, 0.000e+00],
         [0.000e+00, 0.000e+00, 4.630e-02],
         [4.044e-01, 1.210e-02, 1.000e-04],
         [0.000e+00, 0.000e+00, 1.391e-01]])}}

In [86]:
i_vbm = list(el_bs.get_vbm()['band_index'].values())[-1][-1]
i_cbm = list(el_bs.get_cbm()['band_index'].values())[-1][0]
print("VBM:", i_vbm, "CBM:", i_cbm)

VBM: 12 CBM: 13


# Phonon bandstructure

In [87]:
fig_ph_bs = get_plot_bs(ph_bs)
fig_ph_bs.update_yaxes(rangemode="tozero")
fig_ph_bs.show()

In [88]:
fig_ph_dos = get_plot_dos(ph_dos)
fig_ph_dos.show()

In [89]:
fig_ph_bs_and_dos = get_plot_bs_and_dos(ph_bs, ph_dos)
fig_ph_bs_and_dos.update_yaxes(rangemode="tozero")
fig_ph_bs_and_dos.show()

# Distances in the Brillouin zone

In [90]:
xvals = fig_el_bs.to_dict()["data"][0]["x"]
for i_branch, branch in enumerate(el_bs.branches):
    wavevectors = get_branch_wavevectors(el_bs, i_branch)
    v0 = np.dot(np.transpose(el_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[0])
    v1 = np.dot(np.transpose(el_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[1])
    print("{0:3d} {1:9.6f} {2:9.6f}".format(i_branch, np.linalg.norm(v1-v0), xvals[branch["end_index"]]-xvals[branch["start_index"]]))

  0  0.985356  0.985356
  1  0.492678  0.492678
  2  0.348376  0.348376
  3  1.045128  1.045128
  4  0.853344  0.853344
  5  0.603405  0.603405
  6  0.348376  0.348376
  7  0.696752  0.696752
  8  0.603405  0.603405
  9  0.348376  0.348376


In [91]:
xvals = fig_ph_bs.to_dict()["data"][0]["x"]
for i_branch, branch in enumerate(ph_bs.branches):
    wavevectors = get_branch_wavevectors(ph_bs, i_branch)
    v0 = np.dot(np.transpose(ph_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[0])
    v1 = np.dot(np.transpose(ph_bs.structure.lattice.reciprocal_lattice.matrix), wavevectors[1])
    print("{0:3d} {1:9.6f} {2:9.6f}".format(i_branch, np.linalg.norm(v1-v0), xvals[branch["end_index"]]-xvals[branch["start_index"]]))

  0  0.998310  0.998310
  1  0.499155  0.499155
  2  0.352956  0.352956
  3  1.058867  1.058867
  4  0.864562  0.864562
  5  0.611337  0.611337
  6  0.352956  0.352956
  7  0.705912  0.705912
  8  0.611337  0.611337
  9  0.352956  0.352956


In [92]:
el_bs.structure

Structure Summary
Lattice
    abc : 4.508910230856454 4.508910230856454 4.508910230856454
 angles : 60.00000000000001 60.00000000000001 60.00000000000001
 volume : 64.81861825239439
      A : 0.0 3.188281 3.188281
      B : 3.188281 0.0 3.188281
      C : 3.188281 3.188281 0.0
    pbc : True True True
PeriodicSite: Sc (3.188, 3.188, 3.188) [0.5, 0.5, 0.5]
PeriodicSite: Sb (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Pd (1.594, 1.594, 1.594) [0.25, 0.25, 0.25]

In [93]:
ph_bs.structure

Structure Summary
Lattice
    abc : 4.450405610449663 4.450405610449663 4.450405610449663
 angles : 59.99999999999999 59.99999999999999 59.99999999999999
 volume : 62.32808526548456
      A : 0.0 3.1469119861796138 3.1469119861796138
      B : 3.1469119861796138 0.0 3.1469119861796138
      C : 3.1469119861796138 3.1469119861796138 0.0
    pbc : True True True
PeriodicSite: Sc (3.147, 3.147, 3.147) [0.5, 0.5, 0.5]
PeriodicSite: Sb (3.147, 3.147, 0.0) [0.0, 0.0, 1.0]
PeriodicSite: Pd (1.573, 1.573, 1.573) [0.25, 0.25, 0.25]

In [94]:
prim_struc

Structure Summary
Lattice
    abc : 4.490139293110916 4.490138800654892 4.49013962
 angles : 59.99999396372361 59.999997591744204 59.99999914753195
 volume : 64.0124422932965
      A : 3.8885746 -0.0 2.24506981
      B : 1.29619119 3.66618281 2.24506981
      C : 0.0 0.0 4.49013962
    pbc : True True True
PeriodicSite: Sc (2.592, 1.833, 4.49) [0.5, 0.5, 0.5]
PeriodicSite: Sb (0.0, 0.0, 0.0) [0.0, 0.0, -0.0]
PeriodicSite: Pd (1.296, 0.9165, 2.245) [0.25, 0.25, 0.25]

# Specific heat

In [95]:
temperatures = np.arange(0,1000,5)
R = 8.314
nat = len(prim_struc)
ph_cv = np.array([ph_dos.cv(temperatures[i]) for i in range(len(temperatures))])/(3*nat*R)

In [96]:
fig = go.Figure()
scatter = go.Scatter(x=temperatures, y=ph_cv)
fig.add_trace(scatter)
fig.add_hline(y=1, line_width=2, line_color="red")
fig.update_layout(
    xaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
    yaxis =  {'mirror': True, 'showgrid': False, 'ticks': 'inside', 'ticklen':10},
    xaxis_title = "Temperature",
    yaxis_title = "$C_v\,/\,3N_{\!at}R$",        
)
fig.show()